# Simple chatbot using Hugging Face Transformers library! 

Try it talking to a chatbot through the GUI!

**How it works:**
1. Run the cell and wait for the tkinter window to pop out
2. Start chatting!

**Required modules:**
- `pip install transformers`
- `pip install torch`

**Optional modules:**
- `pip install huggingface_hub[hf_xet]` for better model loading

In [3]:
import threading
import tkinter as tk
from tkinter import Entry, scrolledtext, Button, Label
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re

# nicer GUI + conversation support (multiple messages)
window = tk.Tk()
window.title("WebIdeasBot")
window.geometry("800x520")

chat_window = scrolledtext.ScrolledText(window, width=95, height=24, state=tk.DISABLED, wrap=tk.WORD)
chat_window.configure(font=("Consolas", 11))
chat_window.pack(padx=8, pady=(8, 4), fill=tk.BOTH, expand=False)

controls_frame = tk.Frame(window)
controls_frame.pack(fill=tk.X, padx=8, pady=(0,8))

user_input = Entry(controls_frame, width=72, font=("Consolas", 11))
user_input.grid(row=0, column=0, padx=(0,8), sticky="we")
user_input.focus_set()

send_btn = Button(controls_frame, text="Send", width=10)
send_btn.grid(row=0, column=1, padx=(0,8))
clear_btn = Button(controls_frame, text="Clear", width=10)
clear_btn.grid(row=0, column=2)

status_label = Label(window, text="Loading model... (this may take a while)", anchor="w")
status_label.pack(fill=tk.X, padx=8)

tokenizer = None
model = None
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
conversation_history = []  # list of tuples: ("User"/"Bot", text)
lock = threading.Lock()

def append_chat(role, text):
    chat_window.configure(state=tk.NORMAL)
    chat_window.insert(tk.END, f"{role}: {text}\n\n")
    chat_window.see(tk.END)
    chat_window.configure(state=tk.DISABLED)

def load_model():
    global tokenizer, model
    try:
        # Using DialoGPT-medium for better quality responses
        model_name = "microsoft/DialoGPT-medium"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(model_name)
        # Add the EOS token as PAD token to avoid warnings
        tokenizer.pad_token = tokenizer.eos_token
        model.to(device)
        model.eval()
        window.after(0, lambda: _on_model_ready())
    except Exception as e:
        window.after(0, lambda: status_label.config(text=f"Model load error: {e}"))

def _on_model_ready():
    status_label.config(text=f"Model loaded. Device: {device.type}")
    append_chat("System", "Model loaded. You can start chatting.")

threading.Thread(target=load_model, daemon=True).start()

def _is_bad_response(s: str) -> bool:
    s = (s or "").strip()
    if not s:
        return True
    # too short
    if len(s) < 2:
        return True
    # only punctuation / underscores / newlines
    if set(s) <= set(" _-.\n"):
        return True
    # same token repeated many times
    tokens = [t for t in re.split(r"\s+", s) if t]
    if tokens and len(set(tokens)) == 1 and len(tokens) > 3:
        return True
    # check for gibberish (high ratio of consonants to vowels)
    if len(s) > 10:
        vowels = sum(1 for c in s.lower() if c in 'aeiou')
        consonants = sum(1 for c in s.lower() if c.isalpha() and c not in 'aeiou')
        if vowels > 0 and consonants / vowels > 4:  # more than 4 consonants per vowel
            return True
    return False

def _clean_response(s: str) -> str:
    if not s:
        return ""
    # remove leading/trailing whitespace and repeated identical lines
    lines = [ln.rstrip() for ln in s.splitlines() if ln.strip()]
    cleaned = []
    prev = None
    for ln in lines:
        if ln == prev:
            continue
        cleaned.append(ln)
        prev = ln
    # join and trim runaway repeats
    out = "\n".join(cleaned).strip()
    # Remove any unwanted prefixes that the model might generate
    out = re.sub(r"^(User|Human|Bot):\s*", "", out)
    # Remove any URLs or gibberish strings
    out = re.sub(r'http\S+|www\.\S+|\S*[0-9][0-9]\S*', '', out)
    # Remove multiple spaces
    out = re.sub(r'\s+', ' ', out)
    return out.strip()

def generate_response_sync(prompt_text):
    if tokenizer is None or model is None:
        return "[Model still loading]"
    
    try:
        # Encode the input
        inputs = tokenizer.encode(prompt_text, return_tensors="pt", truncation=True, max_length=1024).to(device)
        attention_mask = torch.ones(inputs.shape, dtype=torch.long, device=device)
        
        # Generate response with carefully tuned parameters
        outputs = model.generate(
            inputs,
            attention_mask=attention_mask,
            max_new_tokens=50,  # Keep responses concise
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.7,  # More focused responses
            top_p=0.9,  # Nucleus sampling
            top_k=40,  # Limit vocabulary diversity
            repetition_penalty=1.3,  # Prevent repetition
            no_repeat_ngram_size=3,  # Prevent repetition of phrases
            early_stopping=True
        )
        
        # Decode only the new tokens
        response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
        response = _clean_response(response)
        
        if not _is_bad_response(response):
            return response
        
        # If first attempt failed, try with different parameters
        outputs = model.generate(
            inputs,
            attention_mask=attention_mask,
            max_new_tokens=50,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.6,  # Even more focused
            top_p=0.85,
            top_k=30,
            repetition_penalty=1.4,
            no_repeat_ngram_size=3,
            early_stopping=True
        )
        
        response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
        response = _clean_response(response)
        
        if not _is_bad_response(response):
            return response
            
        return "I'm not sure how to respond to that. Could you try rephrasing your message?"
    except Exception as e:
        return f"[Error generating response: {e}]"

def generate_response_async():
    with lock:
        status_label.config(text="Generating...")
        send_btn.config(state=tk.DISABLED)
        user_input.config(state=tk.DISABLED)

    # Format conversation history for the model
    max_turns = 4  # Reduced from 6 to keep context more focused
    recent = conversation_history[-max_turns*2:]
    prompt_parts = []
    for role, msg in recent:
        if role == "User":
            prompt_parts.append(msg)
    prompt_text = " ".join(prompt_parts[-2:])  # Only use last 2 user messages for context

    response = generate_response_sync(prompt_text)

    # update UI back on main thread
    def finish():
        with lock:
            append_chat("WebIdeasBot", response)
            conversation_history.append(("Bot", response))
            status_label.config(text="Ready")
            send_btn.config(state=tk.NORMAL)
            user_input.config(state=tk.NORMAL)
            user_input.focus_set()
    window.after(0, finish)

def on_send(event=None):
    text = user_input.get().strip()
    if not text:
        return
    append_chat("You", text)
    conversation_history.append(("User", text))
    user_input.delete(0, tk.END)
    # start generation in background
    threading.Thread(target=generate_response_async, daemon=True).start()

def on_clear():
    global conversation_history
    conversation_history = []
    chat_window.configure(state=tk.NORMAL)
    chat_window.delete("1.0", tk.END)
    chat_window.configure(state=tk.DISABLED)
    status_label.config(text="Cleared conversation. Model ready." if model is not None else "Cleared conversation.")

send_btn.config(command=on_send)
clear_btn.config(command=on_clear)
user_input.bind("<Return>", on_send)

window.mainloop()

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

c:\Anaconda\envs\sideprojectenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gerra\.cache\huggingface\hub\models--microsoft--DialoGPT-medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]